# Vilar Oscillator
***
**From:** Vilar, José M. G. et al. “Mechanisms of noise-resistance in genetic oscillators.” PNAS, vol. 99 no. 9, 2002, pp. 5988-5992., doi.org/10.1073/pnas.092133899.
***
## Setup the Environment
***

In [ ]:
import numpy
from dask.distributed import Client

In [ ]:
import gillespy2

In [ ]:
from sciope.utilities.gillespy2 import wrapper
from sciope.designs import latin_hypercube_sampling
from sciope.utilities.summarystats.auto_tsfresh import SummariesTSFRESH
from sciope.stochmet.stochmet import StochMET
from sciope.models.label_propagation import LPModel

***
## Create the Vilar Oscillator Model
***

In [ ]:
def create_vilar_oscillator(parameter_values=None):
    model = gillespy2.Model(name="Vilar_Oscillator")
    model.volume = 1

    # Variables
    Da = gillespy2.Species(name="Da", initial_value=1, mode="discrete")
    Da_prime = gillespy2.Species(name="Da_prime", initial_value=0, mode="discrete")
    Ma = gillespy2.Species(name="Ma", initial_value=0, mode="discrete")
    Dr = gillespy2.Species(name="Dr", initial_value=1, mode="discrete")
    Dr_prime = gillespy2.Species(name="Dr_prime", initial_value=0, mode="discrete")
    Mr = gillespy2.Species(name="Mr", initial_value=0, mode="discrete")
    C = gillespy2.Species(name="C", initial_value=0, mode="discrete")
    A = gillespy2.Species(name="A", initial_value=0, mode="discrete")
    R = gillespy2.Species(name="R", initial_value=0, mode="discrete")
    model.add_species([Da, Da_prime, Ma, Dr, Dr_prime, Mr, C, A, R])

    # Parameters
    alphaA = gillespy2.Parameter(name="alphaA", expression="50")
    alphaA_prime = gillespy2.Parameter(name="alphaA_prime", expression="500")
    alphaR = gillespy2.Parameter(name="alphaR", expression="0.01")
    alphaR_prime = gillespy2.Parameter(name="alphaR_prime", expression="50")
    betaA = gillespy2.Parameter(name="betaA", expression="50")
    betaR = gillespy2.Parameter(name="betaR", expression="5")
    deltaMA = gillespy2.Parameter(name="deltaMA", expression="10")
    deltaMR = gillespy2.Parameter(name="deltaMR", expression="0.5")
    deltaA = gillespy2.Parameter(name="deltaA", expression="1")
    deltaR = gillespy2.Parameter(name="deltaR", expression="0.2")
    gammaA = gillespy2.Parameter(name="gammaA", expression="1")
    gammaR = gillespy2.Parameter(name="gammaR", expression="1")
    gammaC = gillespy2.Parameter(name="gammaC", expression="2")
    thetaA = gillespy2.Parameter(name="thetaA", expression="50")
    thetaR = gillespy2.Parameter(name="thetaR", expression="100")
    model.add_parameter([
        alphaA, alphaA_prime, alphaR, alphaR_prime, betaA, betaR, deltaMA,
        deltaMR, deltaA, deltaR, gammaA, gammaR, gammaC, thetaA, thetaR
    ])

    # Reactions
    r1 = gillespy2.Reaction(
        name="r1", rate="gammaC",
        reactants={'A': 1, 'R': 1}, products={'C': 1}
    )
    r2 = gillespy2.Reaction(
        name="r2", rate="deltaA",
        reactants={'A': 1}, products={}
    )
    r3 = gillespy2.Reaction(
        name="r3", rate="deltaA",
        reactants={'C': 1}, products={'R': 1}
    )
    r4 = gillespy2.Reaction(
        name="r4", rate="deltaR",
        reactants={'R': 1}, products={}
    )
    r5 = gillespy2.Reaction(
        name="r5", rate="gammaA",
        reactants={'A': 1, 'Da': 1}, products={'Da_prime': 1}
    )
    r6 = gillespy2.Reaction(
        name="r6", rate="thetaA",
        reactants={'Da_prime': 1}, products={'A': 1, 'Da': 1}
    )
    r7 = gillespy2.Reaction(
        name="r7", rate="alphaA",
        reactants={'Da': 1}, products={'Da': 1, 'Ma': 1}
    )
    r8 = gillespy2.Reaction(
        name="r8", rate="alphaA_prime",
        reactants={'Da_prime': 1}, products={'Da_prime': 1, 'Ma': 1}
    )
    r9 = gillespy2.Reaction(
        name="r9", rate="deltaMA",
        reactants={'Ma': 1}, products={}
    )
    r10 = gillespy2.Reaction(
        name="r10", rate="betaA",
        reactants={'Ma': 1}, products={'A': 1, 'Ma': 1}
    )
    r11 = gillespy2.Reaction(
        name="r11", rate="gammaR",
        reactants={'A': 1, 'Dr': 1}, products={'Dr_prime': 1}
    )
    r12 = gillespy2.Reaction(
        name="r12", rate="thetaR",
        reactants={'Dr_prime': 1}, products={'A': 1, 'Dr': 1}
    )
    r13 = gillespy2.Reaction(
        name="r13", rate="alphaR",
        reactants={'Dr': 1}, products={'Dr': 1, 'Mr': 1}
    )
    r14 = gillespy2.Reaction(
        name="r14", rate="alphaR_prime",
        reactants={'Dr_prime': 1}, products={'Dr_prime': 1, 'Mr': 1}
    )
    r15 = gillespy2.Reaction(
        name="r15", rate="deltaMR",
        reactants={'Mr': 1}, products={}
    )
    r16 = gillespy2.Reaction(
        name="r16", rate="betaR",
        reactants={'Mr': 1}, products={'Mr': 1, 'R': 1}
    )
    model.add_reaction([r1, r2, r3, r4, r5, r6, r7, r8, r9, r10, r11, r12, r13, r14, r15, r16])

    # Timespan
    tspan = gillespy2.TimeSpan.arange(1, t=200)
    model.timespan(tspan)
    return model

### Instantiate the Model

In [ ]:
model = create_vilar_oscillator()

***
## Simulation Parameters
***

In [ ]:
def configure_simulation():
    solver = gillespy2.SSACSolver(model=model)
    kwargs = {
        "solver":solver,
        "number_of_trajectories":100,
        # "seed":None,
        # "tau_tol":0.03,
        # "integrator_options":{'rtol': 0.001, 'atol': 1e-06},
    }
    return kwargs

***
## Model Exploration
***
### Define simulator function (using gillespy2 wrapper)

In [ ]:
settings = configure_simulation()
simulator = wrapper.get_simulator(gillespy_model=model, run_settings=settings, species_of_interest=['Da', 'Da_prime', 'Ma', 'Dr', 'Dr_prime', 'Mr', 'C', 'A', 'R'])
expression_array = wrapper.get_parameter_expression_array(model)

### Start local cluster using dask client

In [ ]:
c = Client()
c

### Define parameter sampler/design and summary statistics

In [ ]:
lhc = latin_hypercube_sampling.LatinHypercube(xmin=expression_array, xmax=expression_array*3)
lhc.generate_array(1000) # creates a LHD of size 1000

# will use default minimal set of features
summary_stats = SummariesTSFRESH()

### Initiate StochMET

In [ ]:
met = StochMET(simulator, lhc, summary_stats)

***
## Run parameter sweep
***

In [ ]:
met.compute(n_points=500, chunk_size=10)

***
## Explore the result
***
First lets add some appropiate information about the model and features

In [ ]:
met.data.configurations['listOfParameters'] = list(model.listOfParameters.keys())
met.data.configurations['listOfSpecies'] = list(model.listOfSpecies.keys())
met.data.configurations['listOfSummaries'] = met.summaries.features
met.data.configurations['timepoints'] = model.tspan

Here we use UMAP for dimension reduction

In [ ]:
met.explore(dr_method='umap')

Here lets use the dimension reduction embedding as input data

In [ ]:
data = met.dr_model.embedding_

model_lp = LPModel()
# train using basinhopping
model_lp.train(data, met.data.user_labels, min_=0.01, max_=10, niter=50)

Just to vislualize the result we will map the label distribution to the user_labels (will enable us to see the LP model output when using method 'explore')

In [ ]:
user_labels = numpy.copy(met.data.user_labels)
# takes the label corresponding to index 0
met.data.user_labels = model_lp.model.label_distributions_[:, 0]

In [ ]:
met.explore(dr_method='umap')

In [ ]:
met.data.user_labels = user_labels